In [18]:
import pandas as pd
import re
import os
import numpy as np
import datetime

In [19]:
csv_list = []

LUIGI_LOG_PATH = '/usr/local/trustyou/home/andream/nfs/processmining/pm4pyexample/'
LUIGI_LOG_PATH = '/usr/local/trustyou/home/andream/nfs/processmining/minilogs/daily/logs/'
#LUIGI_LOG_PATH = '/usr/local/trustyou/home/andream/nfs/processmining/minilogs/table-precomp/logs/tch/'
#LUIGI_LOG_PATH='/usr/local/trustyou/home/andream/nfs/processmining/minilogs/table-precomp/logs/october/mongo/'
for filename in os.listdir(LUIGI_LOG_PATH):
    if filename.endswith('.csv'):
        log_path = LUIGI_LOG_PATH+'/'+filename
        csv_list.append(filename)
appended_df = pd.DataFrame()
#appended_df
csv_list.sort()
csv_list

['daily.2019-09-01_09-29-01.csv',
 'daily.2019-09-03_03-44-01.csv',
 'daily.2019-09-04_23-23-01.csv',
 'daily.2019-09-06_20-34-01.csv',
 'daily.2019-09-09_01-34-02.csv',
 'daily.2019-09-11_05-40-02.csv',
 'daily.2019-09-13_04-11-01.csv',
 'daily.2019-09-15_01-41-01.csv',
 'daily.2019-09-16_19-44-01.csv',
 'daily.2019-09-18_14-08-01.csv',
 'daily.2019-09-20_12-49-01.csv',
 'daily.2019-09-22_07-40-01.csv',
 'daily.2019-09-24_05-05-02.csv']

In [20]:
from difflib import SequenceMatcher 
  
def longestSubstring(str1,str2): 
  
     # initialize SequenceMatcher object with  
     # input string 
     seqMatch = SequenceMatcher(None,str1,str2) 
  
     # find match of longest sub-string 
     # output will be like Match(a=0, b=0, size=5) 
     match = seqMatch.find_longest_match(0, len(str1), 0, len(str2)) 
  
     # print longest substring 
     if (match.size!=0): 
          result = str1[match.a: match.a + match.size]
          print (result)  
          return result
     else: 
          print ('No longest common sub-string found')

In [21]:
PROCESS_NAME = longestSubstring(csv_list[0],csv_list[-1])
OUTPUT_CATT_PATH = LUIGI_LOG_PATH+'/../catts_'+str(PROCESS_NAME)+'.csv'
print(OUTPUT_CATT_PATH)

daily.2019-09-
/usr/local/trustyou/home/andream/nfs/processmining/minilogs/daily/logs//../catts_daily.2019-09-.csv


In [22]:
for filename in os.listdir(LUIGI_LOG_PATH):
    if filename.endswith('.csv'):
        log_path = LUIGI_LOG_PATH+'/'+filename
        print('Preprocessing... ', filename)
        df = pd.read_csv(log_path, index_col=0)
        df = df[['timestamp','task','state']]
        df['source_file'] = filename

        df['state'][0]='start'
        df['task'][0]='AllTasks'

        df['state'][len(df)-1]='done'
        df['task'][len(df)-1]='AllTasks'
        
        appended_df = appended_df.append(df)
        continue
    else:
        continue

print(len(appended_df))
appended_df.head()

Preprocessing...  daily.2019-09-04_23-23-01.csv
Preprocessing...  daily.2019-09-09_01-34-02.csv
Preprocessing...  daily.2019-09-16_19-44-01.csv
Preprocessing...  daily.2019-09-20_12-49-01.csv
Preprocessing...  daily.2019-09-24_05-05-02.csv
Preprocessing...  daily.2019-09-11_05-40-02.csv
Preprocessing...  daily.2019-09-13_04-11-01.csv
Preprocessing...  daily.2019-09-06_20-34-01.csv
Preprocessing...  daily.2019-09-18_14-08-01.csv
Preprocessing...  daily.2019-09-15_01-41-01.csv
Preprocessing...  daily.2019-09-03_03-44-01.csv
Preprocessing...  daily.2019-09-22_07-40-01.csv
Preprocessing...  daily.2019-09-01_09-29-01.csv
31469


,timestamp,task,state,source_file
0,2019-09-04 23:23:20,AllTasks,start,daily.2019-09-04_23-23-01.csv
1,2019-09-04 23:35:08,"DumpTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep, sql_filename=daily...",start,daily.2019-09-04_23-23-01.csv
2,2019-09-04 23:35:08,"DumpTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep, sql_filename=regul...",start,daily.2019-09-04_23-23-01.csv
3,2019-09-04 23:35:08,"DumpTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep, sql_filename=repro...",start,daily.2019-09-04_23-23-01.csv
4,2019-09-04 23:35:08,"ProxySetupTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep)\n",start,daily.2019-09-04_23-23-01.csv


In [23]:
def get_task(task_call):
    if isinstance(task_call, float):
        return task_call
    return task_call.split('(')[0]

In [24]:
df = appended_df
df['task_call'] = df['task']
df['task_name'] = np.where(df['task'].notnull(), df['task'].apply(get_task), df['task'])
df = df[['task_name','state','timestamp','task_call', 'source_file']]

df['start_time'] = np.where(df['state']=='start', df['timestamp'], np.nan)
df['end_time'] = np.where(df['state']=='done', df['timestamp'], np.nan)

start_df = df[df['state']=='start']
print('Start entries: ',len(start_df))

end_df = df[df['state']=='done']
print('End entries: ',len(end_df))

df.head()

Start entries:  15729
End entries:  15699


,task_name,state,timestamp,task_call,source_file,start_time,end_time
0,AllTasks,start,2019-09-04 23:23:20,AllTasks,daily.2019-09-04_23-23-01.csv,2019-09-04 23:23:20,NaN
1,DumpTask,start,2019-09-04 23:35:08,"DumpTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep, sql_filename=daily...",daily.2019-09-04_23-23-01.csv,2019-09-04 23:35:08,NaN
2,DumpTask,start,2019-09-04 23:35:08,"DumpTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep, sql_filename=regul...",daily.2019-09-04_23-23-01.csv,2019-09-04 23:35:08,NaN
3,DumpTask,start,2019-09-04 23:35:08,"DumpTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep, sql_filename=repro...",daily.2019-09-04_23-23-01.csv,2019-09-04 23:35:08,NaN
4,ProxySetupTask,start,2019-09-04 23:35:08,"ProxySetupTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep)\n",daily.2019-09-04_23-23-01.csv,2019-09-04 23:35:08,NaN


In [25]:
tt = pd.merge(start_df, end_df, on=['task_call', 'source_file'], how='outer')
tt = tt[tt['start_time_x'].notnull()]
tt = tt[tt['end_time_y'].notnull()]

tt['case'] = tt['source_file']
tt['activity'] = tt['task_name_x']
tt['start_time'] = tt['start_time_x']
tt['end_time'] = tt['end_time_y']
print(len(tt))

15711


In [26]:
catt = tt[['case', 'activity','start_time','end_time']]

print(len(catt))
catt.head()
catt.to_csv(OUTPUT_CATT_PATH, index = None, header=True)


15711


In [27]:
pd.options.display.max_colwidth = 100
w_call = tt[['case', 'activity','start_time','end_time', 'task_call']]
grouped = w_call.groupby(['task_call'],as_index=False)
print(len(grouped))
#grouped.head()
#grouped = grouped[grouped['activity']!='CrawlTask']
#grouped.size()

15687


In [28]:
def get_duration(start_time, end_time):
    start = datetime.datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')
    end = datetime.datetime.strptime(end_time, '%Y-%m-%d %H:%M:%S')
    duration = abs(end - start)
    return duration
#get_duration(ex['timestamp'][10],ex['timestamp'][1])

In [29]:
w_duration = tt.copy()
w_duration = w_duration[['case', 'activity','start_time','end_time']]
w_duration['duration'] = w_duration.apply(lambda row: str(get_duration(row['start_time'],row['end_time'])), axis=1)
w_duration['t_duration']= w_duration.apply(lambda row: (get_duration(row['start_time'],row['end_time']).total_seconds())/60/60, axis=1)


w_duration = w_duration[['case','activity','start_time','end_time', 'duration', 't_duration']]
print(len(w_duration))
w_duration.head()

15711


,case,activity,start_time,end_time,duration,t_duration
0,daily.2019-09-04_23-23-01.csv,AllTasks,2019-09-04 23:23:20,2019-09-06 20:33:05,"1 day, 21:09:45",45.162500
1,daily.2019-09-04_23-23-01.csv,DumpTask,2019-09-04 23:35:08,2019-09-04 23:36:33,0:01:25,0.023611
2,daily.2019-09-04_23-23-01.csv,DumpTask,2019-09-04 23:35:08,2019-09-05 00:00:34,0:25:26,0.423889
3,daily.2019-09-04_23-23-01.csv,DumpTask,2019-09-04 23:35:08,2019-09-04 23:35:58,0:00:50,0.013889
4,daily.2019-09-04_23-23-01.csv,ProxySetupTask,2019-09-04 23:35:08,2019-09-04 23:51:32,0:16:24,0.273333


In [30]:
top_long_tasks = w_duration.head(5000)

In [31]:
max_duration = top_long_tasks.groupby(['activity'], sort=False)['t_duration'].max().reset_index()
#.reset_index(name='max_duration').sort_values(by=['max_duration'], ascending=False)
counts = top_long_tasks.groupby(['activity']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
act_group = pd.merge(counts, max_duration, on=['activity'], how='outer')
act_group = act_group.sort_values(by=['counts'], ascending = False)
print(len(act_group))
act_group

67


,activity,counts,t_duration
0,ExtractTask,2014,1.964722
1,CrawlTask,1992,6.859167
2,DumpTask,189,3.421111
3,ConvertDumpTask,166,0.354444
4,GroupTask,31,22.866944
5,QCValidateTask,24,0.098333
6,ExtractChunkTask,24,0.052500
7,CrawlChunkTask,24,0.045833
32,UserDemographicsFullUpdateTask,12,0.000833
26,StripPIITask,12,0.025833


In [32]:
csv_list = []
for filename in os.listdir(LUIGI_LOG_PATH+'../'):
    if filename.endswith('.csv'):
        print(filename)

catts_daily.2019-09-.csv
catts_daily.2019-09-01_09-29-01.csv


In [17]:
csv_list = []
appended_df = pd.DataFrame()
LUIGI_LOG_PATH='/usr/local/trustyou/home/andream/nfs/processmining/minilogs/table-precomp/logs/october/'
#LUIGI_LOG_PATH = '/usr/local/trustyou/home/andream/nfs/processmining/minilogs/daily/logs/'
#LUIGI_LOG_PATH = '/usr/local/trustyou/home/andream/nfs/processmining/minilogs/table-precomp/logs/tch/'
for filename in os.listdir(LUIGI_LOG_PATH):
    if filename.endswith('.csv') and filename.startswith('catts'):
        log_path = LUIGI_LOG_PATH+'/'+filename
        csv_list.append(filename)
        df = pd.read_csv(log_path, index_col=False)
        appended_df = appended_df.append(df)

#appended_df
csv_list.sort()
print(csv_list)
print(len(appended_df))
appended_df.head()

['catts_group.hotel.2019-10-.csv']
7128


,case,activity,start_time,end_time
0,group.hotel.2019-10-15_04-18-01_01.mongo.csv,AllTasks,2019-10-16 02:07:56,2019-10-16 23:21:42
1,group.hotel.2019-10-15_04-18-01_01.mongo.csv,CategorySentimentTask,2019-10-16 02:08:18,2019-10-16 02:08:19
2,group.hotel.2019-10-15_04-18-01_01.mongo.csv,MatchIncDumpTask,2019-10-16 02:08:18,2019-10-16 02:08:48
3,group.hotel.2019-10-15_04-18-01_01.mongo.csv,MgmtResponseIncDumpTask,2019-10-16 02:08:18,2019-10-16 02:09:51
4,group.hotel.2019-10-15_04-18-01_01.mongo.csv,ReviewIncDumpTask,2019-10-16 02:08:18,2019-10-16 02:08:42
